In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [2]:
dataset = pd.read_csv("50_Startups.csv")

In [3]:
dataset.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


## Y a-t-il des valeurs manquantes ?

In [4]:
dataset.isnull().sum()

R&D Spend          0
Administration     0
Marketing Spend    0
State              0
Profit             0
dtype: int64

## Diviser X et Y

In [5]:
X = dataset.loc [:, ['R&D Spend', 'Administration', 'Marketing Spend', 'State']].values

In [6]:
Y = dataset.loc[:, 'Profit'].values

## Encodage des variables de catégories

In [7]:
labelencoder_X = LabelEncoder()

In [8]:
X[:, 3] = labelencoder_X.fit_transform(X[:, 3])

In [9]:
onehotencoder_X = OneHotEncoder(categorical_features=[3])

In [10]:
X = onehotencoder_X.fit_transform(X).toarray()

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


## Piège des variables factices

In [11]:
X = X[:, 1:]

## Alternative method to get dummies

In [12]:
# pd.get_dummies(dataset, columns=["State"]).shape

In [13]:
# pd.get_dummies(dataset, columns=["State"], drop_first=True).shape

## Split dataset in train and test set

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

## Feature Scaling AKA Normaliser avec standard scaler

In [16]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()

sc_X.fit(X_train)

X_train = sc_X.transform(X_train)
X_test = sc_X.transform(X_test)

In [17]:
#
X_train.std(axis=0)

array([1., 1., 1., 1., 1.])

## On effectue la regression

In [18]:
from sklearn.linear_model import LinearRegression

In [19]:
# Si on met fit_intercept on insère une valeur b
regressor = LinearRegression(fit_intercept=True)

In [20]:
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

## Predict

In [21]:
y_pred = regressor.predict(X_test)
y_pred # correspond aux lignes pour lesquelles on ne lui a pas donnée de réponse

array([103015.20159796, 132582.27760816, 132447.73845174,  71976.09851258,
       178537.48221055, 116161.24230165,  67851.69209676,  98791.73374687,
       113969.43533012, 167921.0656955 ])

## Verify

In [22]:
from sklearn.metrics import mean_squared_error

In [23]:
# Moyenne des erreurs au carré

mse = mean_squared_error(y_pred, y_test)

In [25]:
from sklearn.metrics import mean_squared_error
print(np.sqrt(mse))

9137.99015279494


# Stats Models for backward elimination

Stats models n'a pas de valeur b. Il faut donc rajouter une colonne.
L'intérêt de la backward elimination, c'est d'enlever des colonnes, tant que la valeur R-squared ne diminue pas (ces features ne sont pas significatives voir rajoutent du bruit)

In [ ]:
# import statsmodels from 

## R^2

Est-ce que mon modèle est meilleur que la moyenne ?

MSE est utilisé sur le test alors que R^2 est utilisé sur le train set => plus est elevé (car c'est 1 - ... ) et mieux c'est. Si on enlève une feature et le R^2 augmente, cela veut dire que ct une source de bruit.

## R^2_adj

Il va augmenter uniquement si la varibale ajoutée contrebalance le biais introduit par l'ajout d'une variable